In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import matplotlib.mlab   as mlab

from matplotlib import rc
from kernel_two_sample_test import kernel_two_sample_test
from kernel_two_sample_test_nonuniform import kernel_two_sample_test_nonuniform
from sklearn.metrics import pairwise_distances

from scipy.spatial.distance import cdist
from scipy.special import expit
from scipy.stats import bernoulli
from numpy.polynomial.polynomial import polyval

rc('font', **{'family': 'serif', 'serif': ['Modern Computer']})
rc('text', usetex=True)

%matplotlib inline

In [ ]:
# generate data from the marginal distributions P(X_0) and P(X_1)
ns = 500
d  = 5
X  = np.random.randn(ns,d)

In [ ]:
# generate Y_0 and Y_1 from the conditional models
beta_vec  = np.array([0.1,0.2,0.3,0.4,0.5])
alpha_vec = np.array([0.5,0.4,0.3,0.2,0.1])
alpha_0   = 0.05

## Scenario I

In [ ]:
b = 0

Prob_vec = expit(np.dot(alpha_vec,X.T) + alpha_0)

T  = bernoulli.rvs(Prob_vec)
Y0 = np.dot(beta_vec,X[T==0,:].T) + 0.1*np.random.randn(X[T==0,:].shape[0])
Y1 = np.dot(beta_vec,X[T==1,:].T) + b + 0.1*np.random.randn(X[T==1,:].shape[0])

In [ ]:
# plot the true functions and the samples
fig, ax = plt.subplots(1,1)

ax.plot(Y0,linewidth=1,linestyle="None",marker='o',color='red',label="Y_0")
ax.plot(Y1,linewidth=1,linestyle="None",marker='x',color='blue',label="Y_1")
ax.legend(["$Y_0$","$Y_1$"])
ax.set_title("Scenario I", fontsize=16)
ax.set_ylabel("Y", fontsize=16)

fig.tight_layout()

In [ ]:
# save the figure
fig.savefig('data_scenario_1.eps',dpi=100)

In [ ]:
# plot the histograms
fig, ax = plt.subplots(1,1)
n, bins, patches = ax.hist(Y0,30,facecolor='green',weights=Prob_vec[T==0],density=True,alpha=0.6,label="$Y_0$")
#y0 = mlab.normpdf(bins, 0, 1)
y0 = scipy.stats.norm.pdf(bins, 0, 1) 
#l = plt.plot(bins, y0, 'r--', linewidth=3,label='Y0')

n, bins, patches = ax.hist(Y1,30,facecolor='blue',weights=Prob_vec[T==1],density=True,alpha=0.6,label="$Y_1$")
#y1 = mlab.normpdf(bins, 0, 1)
y1 = scipy.stats.norm.pdf(bins, 0, 1) 
#l = plt.plot(bins, y1, 'm--', linewidth=3,label='Y1')

ax.legend()
ax.set_title('Scenario I', fontsize=16)
ax.set_ylabel('P(Y)', fontsize=16)
ax.set_xlabel('Y', fontsize=16)

fig.tight_layout()

In [ ]:
# save the figure
fig.savefig('data_hist_scenario_1.pdf',dpi=100)

## Perform Two-sample Testing

In [ ]:
YY0 = Y0[:,np.newaxis]
YY1 = Y1[:,np.newaxis]

# Gaussian RBF kernel
sigma2 = np.median(pairwise_distances(YY0, YY1, metric='euclidean'))**2
mmd2u_rbf, mmd2u_null_rbf, p_value_rbf = kernel_two_sample_test_nonuniform(YY0, YY1, Prob_vec,
                                                    kernel_function='rbf',
                                                    gamma=1.0/sigma2,
                                                    verbose=True)

# Linear kernel
mmd2u_lin, mmd2u_null_lin, p_value_lin = kernel_two_sample_test_nonuniform(YY0, YY1, Prob_vec,
                                                    kernel_function='linear',
                                                    verbose=True)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,4))

prob, bins, patches = ax[0].hist(mmd2u_null_rbf, bins=50)
ax[0].plot(mmd2u_rbf, prob.max()/30, 'w*', markersize=24, markeredgecolor='k',
         markeredgewidth=2, label="$MMD^2_u = %s$" % mmd2u_rbf)
ax[0].set_xlabel('$MMD^2_u$')
ax[0].set_ylabel('$p(MMD^2_u)$')
ax[0].legend(numpoints=1)
ax[0].set_title('$MMD^2_u$: null-distribution and observed value. $p$-value=%s'
        % p_value_rbf)

#
prob, bins, patches = ax[1].hist(mmd2u_null_lin, bins=50)
ax[1].plot(mmd2u_lin, prob.max()/30, 'w*', markersize=24, markeredgecolor='k',
         markeredgewidth=2, label="$MMD^2_u = %s$" % mmd2u_lin)
ax[1].set_xlabel('$MMD^2_u$')
ax[1].set_ylabel('$p(MMD^2_u)$')
ax[1].legend(numpoints=1)
ax[1].set_title('$MMD^2_u$: null-distribution and observed value. $p$-value=%s'
        % p_value_lin)

fig.tight_layout()

## Scenario II

In [ ]:
b = 2

Prob_vec = expit(np.dot(alpha_vec,X.T) + alpha_0)

T  = bernoulli.rvs(Prob_vec)
Y0 = np.dot(beta_vec,X[T==0,:].T) + 0.1*np.random.randn(X[T==0,:].shape[0])
Y1 = np.dot(beta_vec,X[T==1,:].T) + b + 0.1*np.random.randn(X[T==1,:].shape[0])

In [ ]:
# plot the true functions and the samples
fig,ax = plt.subplots(1,1)

ax.plot(Y0,linewidth=1,color='red',label="Y_0")
ax.plot(Y1,linewidth=1,color='blue',label="Y_1")
ax.legend(["$Y_0$","$Y_1$"])
ax.set_title("Scenario II")
ax.set_ylabel("Y")

fig.tight_layout()

In [ ]:
# save the figure
fig.savefig('data_scenario_2.eps',dpi=100)

In [ ]:
# plot the histograms
fig, ax = plt.subplots(1,1)
n, bins, patches = ax.hist(Y0,30,facecolor='green',weights=Prob_vec[T==0],density=True,alpha=0.6,label="$Y_0$")
#y0 = mlab.normpdf(bins, 0, 1)
y0 = scipy.stats.norm.pdf(bins, 0, 1)
#l = plt.plot(bins, y0, 'r--', linewidth=3,label='Y0')

n, bins, patches = ax.hist(Y1,30,facecolor='blue',weights=Prob_vec[T==1],density=True,alpha=0.6,label="$Y_1$")
#y1 = mlab.normpdf(bins, 2, 1)
y1 = scipy.stats.norm.pdf(bins, 2, 1) 
#l = plt.plot(bins, y1, 'm--', linewidth=3,label='Y1')

ax.legend()
ax.set_title('Scenario II', fontsize=16)
ax.set_ylabel('P(Y)', fontsize=16)
ax.set_xlabel('Y', fontsize=16)

fig.tight_layout()

In [ ]:
# save the figure
fig.savefig('data_hist_scenario_2.pdf',dpi=100)

## Perform Two-sample Testing

In [ ]:
YY0 = Y0[:,np.newaxis]
YY1 = Y1[:,np.newaxis]

sigma2 = np.median(pairwise_distances(YY0, YY1, metric='euclidean'))**2
mmd2u_rbf, mmd2u_null_rbf, p_value_rbf = kernel_two_sample_test(YY0, YY1,
                                                    kernel_function='rbf',
                                                    gamma=1.0/sigma2,
                                                    verbose=True)
    
mmd2u_lin, mmd2u_null_lin, p_value_lin = kernel_two_sample_test(YY0, YY1,
                                                    kernel_function='linear',
                                                    verbose=True)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,4))

prob, bins, patches = ax[0].hist(mmd2u_null_rbf, bins=50, density=True)
ax[0].plot(mmd2u_rbf, prob.max()/30, 'w*', markersize=24, markeredgecolor='k',
         markeredgewidth=2, label="$MMD^2_u = %s$" % mmd2u_rbf)
ax[0].set_xlabel('$MMD^2_u$')
ax[0].set_ylabel('$p(MMD^2_u)$')
ax[0].legend(numpoints=1)
ax[0].set_title('$MMD^2_u$: null-distribution and observed value. $p$-value=%s'
        % p_value_rbf)

#
prob, bins, patches = ax[1].hist(mmd2u_null_lin, bins=50, density=True)
ax[1].plot(mmd2u_lin, prob.max()/30, 'w*', markersize=24, markeredgecolor='k',
         markeredgewidth=2, label="$MMD^2_u = %s$" % mmd2u_lin)
ax[1].set_xlabel('$MMD^2_u$')
ax[1].set_ylabel('$p(MMD^2_u)$')
ax[1].legend(numpoints=1)
ax[1].set_title('$MMD^2_u$: null-distribution and observed value. $p$-value=%s'
        % p_value_lin)

fig.tight_layout()

## Scenario III

In [ ]:
b = 2

Prob_vec = expit(np.dot(alpha_vec,X.T) + alpha_0)

T  = bernoulli.rvs(Prob_vec)
Z  = bernoulli.rvs(0.5,size=len(T[T==1]))

Y0 = np.dot(beta_vec,X[T==0,:].T) + 0.1*np.random.randn(X[T==0,:].shape[0])
Y1 = np.dot(beta_vec,X[T==1,:].T) + (2*Z - 1) + 0.1*np.random.randn(X[T==1,:].shape[0])

In [ ]:
# plot the true functions and the samples
fig,ax = plt.subplots(1,1)

ax.plot(Y0,linewidth=1,color='red',label="Y_0")
ax.plot(Y1,linewidth=1,color='blue',label="Y_1")
ax.legend(["$Y_0$","$Y_1$"])
ax.set_title("Scenario III")
ax.set_ylabel("Y")

fig.tight_layout()

In [ ]:
# save the figure
fig.savefig('data_scenario_3.eps',dpi=100)

In [ ]:
# plot the histograms
fig, ax = plt.subplots(1,1)
n, bins, patches = ax.hist(Y0,30,density=True,weights=Prob_vec[T==0],facecolor='green', alpha=0.6, label="$Y_0$")
#y0 = mlab.normpdf(bins, 0, 1)
y0 = scipy.stats.norm.pdf(bins, 0, 1)
#l = plt.plot(bins, y0, 'r--', linewidth=2,label='Y0')

n, bins, patches = ax.hist(Y1,30,density=True,weights=Prob_vec[T==1],facecolor='blue',alpha=0.6,label="$Y_1$")
#y1 = mlab.normpdf(bins, 2, 1)
y1 = scipy.stats.norm.pdf(bins, 2, 1)
#l = plt.plot(bins, y1, 'm--', linewidth=2,label='Y1')

ax.legend()
ax.set_title('Scenario III', fontsize=16)
ax.set_ylabel('P(Y)', fontsize=16)
ax.set_xlabel('Y', fontsize=16)

fig.tight_layout()

In [ ]:
# save the figure
fig.savefig('data_hist_scenario_3.pdf',dpi=100)

## Perform Two-sample Testing

In [ ]:
YY0 = Y0[:,np.newaxis]
YY1 = Y1[:,np.newaxis]

sigma2 = np.median(pairwise_distances(YY0, YY1, metric='euclidean'))**2
mmd2u_rbf, mmd2u_null_rbf, p_value_rbf = kernel_two_sample_test(YY0, YY1,
                                                    kernel_function='rbf',
                                                    gamma=1.0/sigma2,
                                                    verbose=True)
    
mmd2u_lin, mmd2u_null_lin, p_value_lin = kernel_two_sample_test(YY0, YY1,
                                                    kernel_function='linear',
                                                    verbose=True)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,4))

prob, bins, patches = ax[0].hist(mmd2u_null_rbf, bins=50, density=True)
ax[0].plot(mmd2u_rbf, prob.max()/30, 'w*', markersize=24, markeredgecolor='k',
         markeredgewidth=2, label="$MMD^2_u = %s$" % mmd2u_rbf)
ax[0].set_xlabel('$MMD^2_u$')
ax[0].set_ylabel('$p(MMD^2_u)$')
ax[0].legend(numpoints=1)
ax[0].set_title('$MMD^2_u$: null-distribution and observed value. $p$-value=%s'
        % p_value_rbf)

prob, bins, patches = ax[1].hist(mmd2u_null_lin, bins=50, density=True)
ax[1].plot(mmd2u_lin, prob.max()/30, 'w*', markersize=24, markeredgecolor='k',
         markeredgewidth=2, label="$MMD^2_u = %s$" % mmd2u_lin)
ax[1].set_xlabel('$MMD^2_u$')
ax[1].set_ylabel('$p(MMD^2_u)$')
ax[1].legend(numpoints=1)
ax[1].set_title('$MMD^2_u$: null-distribution and observed value. $p$-value=%s'
        % p_value_lin)

fig.tight_layout()

# Multiple Experiments

We repeat the experiments above.

In [2]:
num_experiments = 1000

# generate data from the marginal distributions P(X_0) and P(X_1)
ns = 50
d  = 5
noise_var = 0.1

# generate Y_0 and Y_1 from the conditional models
beta_vec  = np.array([0.1,0.2,0.3,0.4,0.5])
alpha_vec = np.array([0.05,0.04,0.03,0.02,0.01])
alpha_0   = 0.05

significance_level = 0.01

### Scenario I

In [3]:
b = 0
lin_num_rejects = 0
rbf_num_rejects = 0
for n in range(num_experiments):
    
    ### generate data 
    X  = np.random.randn(ns,d)

    Prob_vec = expit(np.dot(alpha_vec,X.T) + alpha_0)
    
    T  = bernoulli.rvs(Prob_vec)
    Y0 = np.dot(beta_vec,X[T==0,:].T) + noise_var*np.random.randn(X[T==0,:].shape[0])
    Y1 = np.dot(beta_vec,X[T==1,:].T) + b + noise_var*np.random.randn(X[T==1,:].shape[0])
    
    ### calculate the test statistics and p-value
    YY0 = Y0[:,np.newaxis]
    YY1 = Y1[:,np.newaxis]

    # Gaussian RBF kernel
    sigma2 = np.median(pairwise_distances(YY0, YY1, metric='euclidean'))**2
    mmd2u_rbf, mmd2u_null_rbf, p_value_rbf = kernel_two_sample_test_nonuniform(YY0, YY1, Prob_vec,
                                                        kernel_function='rbf',
                                                        gamma=1.0/sigma2,
                                                        verbose=False)
    
    rbf_num_rejects += int(p_value_rbf < significance_level)
    
    # linear kernel
    mmd2u_lin, mmd2u_null_lin, p_value_lin = kernel_two_sample_test_nonuniform(YY0, YY1, Prob_vec,
                                                        kernel_function='linear',
                                                        verbose=False)
    
    lin_num_rejects += int(p_value_lin < significance_level)
    

In [4]:
print('%5f %5f' % (lin_num_rejects/num_experiments,rbf_num_rejects/num_experiments))

0.013000 0.012000


### Scenario II

In [5]:
b = 2

lin_num_rejects = 0
rbf_num_rejects = 0

for n in range(num_experiments):
    
    ### generate data 
    X  = np.random.randn(ns,d)
    Prob_vec = expit(np.dot(alpha_vec,X.T) + alpha_0)

    T  = bernoulli.rvs(Prob_vec)
    Y0 = np.dot(beta_vec,X[T==0,:].T) + noise_var*np.random.randn(X[T==0,:].shape[0])
    Y1 = np.dot(beta_vec,X[T==1,:].T) + b + noise_var*np.random.randn(X[T==1,:].shape[0])
    
    ### calculate the test statistics and p-value
    YY0 = Y0[:,np.newaxis]
    YY1 = Y1[:,np.newaxis]

    # Gaussian RBF kernel
    sigma2 = np.median(pairwise_distances(YY0, YY1, metric='euclidean'))**2
    mmd2u_rbf, mmd2u_null_rbf, p_value_rbf = kernel_two_sample_test_nonuniform(YY0, YY1, Prob_vec,
                                                        kernel_function='rbf',
                                                        gamma=1.0/sigma2,
                                                        verbose=False)
    
    rbf_num_rejects += int(p_value_rbf < significance_level)
    
    # linear kernel
    mmd2u_lin, mmd2u_null_lin, p_value_lin = kernel_two_sample_test_nonuniform(YY0, YY1, Prob_vec,
                                                        kernel_function='linear',
                                                        verbose=False)
    
    lin_num_rejects += int(p_value_lin < significance_level)
    

In [6]:
print('%5f %5f' % (lin_num_rejects/num_experiments,rbf_num_rejects/num_experiments))

1.000000 1.000000


### Scenario III

In [7]:
b = 2

lin_num_rejects = 0
rbf_num_rejects = 0

for n in range(num_experiments):
    
    ### generate data 
    X  = np.random.randn(ns,d)
    Prob_vec = expit(np.dot(alpha_vec,X.T) + alpha_0)

    T  = bernoulli.rvs(Prob_vec)
    Z  = bernoulli.rvs(0.5,size=len(T[T==1]))

    Y0 = np.dot(beta_vec,X[T==0,:].T) + noise_var*np.random.randn(X[T==0,:].shape[0])
    Y1 = np.dot(beta_vec,X[T==1,:].T) + (2*Z - 1) + noise_var*np.random.randn(X[T==1,:].shape[0])
    
    ### calculate the test statistics and p-value
    YY0 = Y0[:,np.newaxis]
    YY1 = Y1[:,np.newaxis]

    # Gaussian RBF kernel
    sigma2 = np.median(pairwise_distances(YY0, YY1, metric='euclidean'))**2
    mmd2u_rbf, mmd2u_null_rbf, p_value_rbf = kernel_two_sample_test_nonuniform(YY0, YY1, Prob_vec,
                                                        kernel_function='rbf',
                                                        gamma=1.0/sigma2,
                                                        verbose=False)
    
    rbf_num_rejects += int(p_value_rbf < significance_level)
    
    # linear kernel
    mmd2u_lin, mmd2u_null_lin, p_value_lin = kernel_two_sample_test_nonuniform(YY0, YY1, Prob_vec,
                                                        kernel_function='linear',
                                                        verbose=False)
    
    lin_num_rejects += int(p_value_lin < significance_level)
    

In [8]:
print('%5f %5f' % (lin_num_rejects/num_experiments,rbf_num_rejects/num_experiments))

0.012000 0.224000
